## Input Files

In [110]:
%pip install plotly
%pip install kaleido

You should consider upgrading via the '/Users/jarred.luzada/.asdf/installs/python/3.10.0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/jarred.luzada/.asdf/installs/python/3.10.0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [111]:
import plotly.express as px
import pandas as pd

writes_throughput_data_path = "logs/benchmark/benchmark_LocationDb_Locations_RTree_CorrelatedPrefix_10000_latest.log"
writes_io_data_path = "logs/disk/disk_cc_1745867829.log"
reads_throughput_data_path = ""
reads_io_data_path = ""

benchmark_type = "insert-query-correlated-prefix"

In [112]:
# Define a function to parse the log file
def parse_io_log_file(file_path):
    # Open and read the log file
    with open(file_path, 'r') as file:
        log_data = file.read()
    
    # Split the log data into individual records using "---" as the delimiter
    records = log_data.split('---')
    
    # Remove any empty records (e.g., due to trailing "---")
    records = [record.strip() for record in records if record.strip()]
    
    # Parse each record into a dictionary
    parsed_records = []
    for record in records:
        record_dict = {}
        for line in record.split('\n'):
            if ':' in line:  # Assuming key-value pairs are separated by ":"
                key, value = line.split(':', 1)
                record_dict[key.strip()] = value.strip()
        parsed_records.append(record_dict)
    
    # Convert the list of dictionaries into a pandas DataFrame
    columns = ['timestamp', 'rchar', 'wchar', 'syscr', 'syscw', 'read_bytes', 'write_bytes', 'cancelled_write_bytes']
    dtypes = {'timestamp': 'int64', 'rchar': 'int64', 'wchar': 'int64', 'syscr': 'int64', 'syscw': 'int64', 'read_bytes': 'int64', 'write_bytes': 'int64', 'cancelled_write_bytes': 'int64'}
    df = pd.DataFrame(parsed_records, columns=columns).astype(dtypes)
    df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')
    return df

io_df = parse_io_log_file(writes_io_data_path)
io_df

,timestamp,rchar,wchar,syscr,syscw,read_bytes,write_bytes,cancelled_write_bytes
0,2025-04-28 19:17:17,25109629,45881,13002,149,0,61440,0
1,2025-04-28 19:17:18,26318636,102511,14339,237,0,69632,0
2,2025-04-28 19:17:19,26551768,351459,14834,481,0,73728,0
3,2025-04-28 19:17:20,26551768,351459,14834,481,0,73728,0
4,2025-04-28 19:17:21,26551768,351459,14834,481,0,73728,0
...,...,...,...,...,...,...,...,...
343,2025-04-28 19:23:11,444150367,981418603,2487858,1238158,0,217722880,11177984
344,2025-04-28 19:23:12,444150367,981418603,2487858,1238158,0,217722880,11177984
345,2025-04-28 19:23:13,444150371,981418603,2487859,1238158,0,217722880,11177984
346,2025-04-28 19:23:14,444150371,981418603,2487859,1238158,0,217722880,11177984


In [113]:
io_df.to_csv(f"./results/{benchmark_type}-io.csv", index=False)

In [114]:
def visualize_io_data(df, ycolumn='rchar', yaxis_title='Characters Read', title='Characters Read vs Timestamp'):
    fig = px.line(
        df,
        x='timestamp',
        y=ycolumn,
        title=title,
        labels={'timestamp': 'Timestamp', ycolumn: ycolumn},
        template='plotly_white'
    )

    # Customize the layout
    fig.update_layout(
        xaxis_title='Timestamp',
        yaxis_title=yaxis_title,
        xaxis=dict(tickangle=45),
        title_x=0.5
    )

    # Show the plot
    return fig

In [ ]:
import json
def parse_throughput_log_file(file_path):
    # Open and read the log file
    rows = []
    with open(file_path, 'r') as file:
        for line in file:
            timestamp, status, message = line.split(" - ")
            json_message = json.loads(message)
            # if json_message["status"] != "success" or json_message["event"] != "command.query":
            #     continue
            metrics = json_message.get("metrics")
            rows.append({
                "timestamp": timestamp,
                #"status": json_message.get("status"),
                "elapsed_time": metrics.get("elapsedTime"),
                "execution_time": metrics.get("executionTime"),
                "compile_time": metrics.get("compileTime"),
                "queue_wait_time": metrics.get("queueWaitTime"),
                # "resultCount": metrics.get("resultCount"),
                # "result_size": metrics.get("resultSize"),
                # "processed_objects": metrics.get("processedObjects"),
            })
    df = pd.DataFrame(rows)
    duration_columns = ["elapsed_time", "execution_time", "compile_time", "queue_wait_time"]
    for col in duration_columns:
        multiplier = df[col].str.extract(r'(ms|ns|s)$', expand=False).map({'ms': 1, 's': 1000, 'ns': 0.001}).fillna(1)
        df[col] = df[col].str.extract(r'(\d+\.?\d*)', expand=False).astype("double")
        df[col] = df[col] * multiplier
    df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')
    return df
    
            
throughput_df = parse_throughput_log_file(writes_throughput_data_path)
throughput_df.head()

/var/folders/tc/_8xj206d6wb42x7t18pt16880000gr/T/ipykernel_5648/2667553966.py:29: FutureWarning:

The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.



,timestamp,elapsed_time,execution_time,compile_time,queue_wait_time
0,2025-04-28 19:18:05,39893.470185,39152.337518,0.0,0.0
1,2025-04-28 19:18:22,15993.054590,15039.940632,0.0,0.0
2,2025-04-28 19:18:29,6194.760836,5896.414002,0.0,0.0
3,2025-04-28 19:18:34,5304.943085,4828.142086,0.0,0.0
4,2025-04-28 19:18:40,5745.380169,5541.652377,0.0,0.0


In [116]:
throughput_df.to_csv(f"./results/{benchmark_type}-throughput.csv", index=False)

In [ ]:
def visualize_throughput_data(df, resample=False):
    # not sampled on inserts only 
    df_grouped = df.resample('1s', on='timestamp').mean().reset_index() if resample else df

    # Create a line plot for execution time vs timestamp
    fig = px.line(
        df_grouped,
        x='timestamp',
        y='execution_time',
        title='Execution Time vs Timestamp (Bucketed by 1 Second)',
        labels={'timestamp': 'Timestamp', 'execution_time': 'Execution Time (ms)'},
        template='plotly_white'
    )

    # Customize the layout
    fig.update_layout(
        xaxis_title='Timestamp',
        yaxis_title='Execution Time (ms)',
        xaxis=dict(tickangle=45),
        title_x=0.5
    )
    return fig

visualize_throughput_data(throughput_df)

In [118]:
def show_aggregate_stats(df):
    return df.describe([.50, .75, .95, .99])

## Write
- Write Throughput
  - mean
  - std.dev
  - median
  - p95
  - p99
- Bytes Written to Disk
  - per second
  - aggregate

### Write Throughput

In [119]:
show_aggregate_stats(throughput_df[["elapsed_time"]])

,elapsed_time
count,41.000000
mean,7805.418419
std,5803.621936
min,5266.234711
50%,6184.351336
75%,6793.803378
95%,15993.054590
99%,31682.598148
max,39893.470185


### Bytes Written to Disk

#### Aggregate

In [120]:
show_aggregate_stats(io_df[["wchar"]])

,wchar
count,3.480000e+02
mean,4.636151e+08
std,3.283450e+08
min,4.588100e+04
50%,4.942073e+08
75%,7.365819e+08
95%,9.814172e+08
99%,9.814186e+08
max,9.814191e+08


#### As time

In [121]:
visualize_io_data(io_df, ycolumn='wchar', yaxis_title='Characters Written', title='Characters Written vs Timestamp').show()

## Reads
- Read Throughput
  - mean
  - std.dev
  - median
  - p95
  - p99
- Bytes Read from Disk
  - per second
  - aggregate

In [122]:
show_aggregate_stats(io_df[["rchar"]])

,rchar
count,3.480000e+02
mean,2.202524e+08
std,1.437641e+08
min,2.510963e+07
50%,2.392417e+08
75%,3.407820e+08
95%,4.441483e+08
99%,4.441504e+08
max,4.441509e+08


In [123]:
visualize_io_data(io_df, ycolumn='rchar', yaxis_title='Characters Read', title='Characters Read vs Timestamp').show()